In [1]:
import pandas as pd

BASE_PATH = "C:\\Users\\Haroldo Duraes\\Desktop\\GOvGO\\v0\\#DATA\\PNCP\\"
CAT_PATH = BASE_PATH + "CAT\\"
FILE = CAT_PATH + "Lista CATMAT.xlsx"
ABA = "Materiais"

In [2]:
df = pd.read_excel(FILE, sheet_name=ABA)
display(df.head())

,Código do Grupo,Nome do Grupo,Código da Classe,Nome da Classe,Código do PDM,Nome do PDM,Código do Item,Descrição do Item,Código NCM
0,10,ARMAMENTO,1005,ARMAS DE FOGO DE CALIBRE ATÉ 120MM,1712,PEÇAS / ACESSÓRIOS ARMAMENTO,446820,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",-
1,10,ARMAMENTO,1005,ARMAS DE FOGO DE CALIBRE ATÉ 120MM,1712,PEÇAS / ACESSÓRIOS ARMAMENTO,446821,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",-
2,10,ARMAMENTO,1005,ARMAS DE FOGO DE CALIBRE ATÉ 120MM,1712,PEÇAS / ACESSÓRIOS ARMAMENTO,446823,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",-
3,10,ARMAMENTO,1005,ARMAS DE FOGO DE CALIBRE ATÉ 120MM,1712,PEÇAS / ACESSÓRIOS ARMAMENTO,446840,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",-
4,10,ARMAMENTO,1005,ARMAS DE FOGO DE CALIBRE ATÉ 120MM,1712,PEÇAS / ACESSÓRIOS ARMAMENTO,446842,"PEÇAS / ACESSÓRIOS ARMAMENTO, MATERIAL:AÇO, TI...",-


In [5]:
catmat_df.shape

(157853, 4)

In [ ]:
# Criando uma cópia do dataframe original para não modificá-lo
sample_df = catmat_df.copy()

# Adicionando a coluna CATEGORIA como concatenação de GRUPO e CLASSE
sample_df['CATEGORIA'] = sample_df['GRUPO'] + '; ' + sample_df['CLASSE']

# Definindo o número máximo de itens por PDM para cada categoria
n = 5  # Você pode ajustar este valor conforme necessário

# Função para selecionar no máximo n itens por grupo
def sample_group(group):
    if len(group) <= n:
        return group
    else:
        return group.sample(n, random_state=42)  # Usando random_state para reprodutibilidade

# Aplicar a amostragem por CATEGORIA e PDM
sample_df = sample_df.groupby(['CATEGORIA', 'PDM']).apply(sample_group).reset_index(drop=True)

sample_df.shape


C:\Users\Haroldo Duraes\AppData\Local\Temp\ipykernel_33060\2519318170.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df = sample_df.groupby(['CATEGORIA', 'PDM']).apply(sample_group).reset_index(drop=True)


(32803, 5)

In [11]:
# Contar quantos itens existem para cada combinação CATEGORIA+PDM
counts = sample_df.groupby(['CATEGORIA', 'PDM']).size()

# Criar uma coluna que identifica se o grupo CATEGORIA+PDM tem apenas um item
sample_df['single_item_group'] = sample_df.apply(
    lambda x: counts[(x['CATEGORIA'], x['PDM'])] == 1, 
    axis=1
)

# Inicializar os dataframes de treinamento e validação
training_df = pd.DataFrame(columns=['ITEM', 'CATEGORIA'])
validation_df = pd.DataFrame(columns=['ITEM', 'CATEGORIA'])

# Função para separar os itens em treinamento e validação
def split_group(group):
    global training_df, validation_df
    
    # Se o grupo tiver apenas um item
    if group['single_item_group'].iloc[0]:
        # Adicionar o item tanto em treinamento quanto em validação
        item_data = pd.DataFrame({
            'ITEM': [group['ITEM'].iloc[0]],
            'CATEGORIA': [group['CATEGORIA'].iloc[0]]
        })
        training_df = pd.concat([training_df, item_data], ignore_index=True)
        validation_df = pd.concat([validation_df, item_data], ignore_index=True)
    else:
        # Pegar 1 item para treinamento
        training_item = group.iloc[[0]]
        training_data = pd.DataFrame({
            'ITEM': [training_item['ITEM'].iloc[0]],
            'CATEGORIA': [training_item['CATEGORIA'].iloc[0]]
        })
        training_df = pd.concat([training_df, training_data], ignore_index=True)
        
        # Pegar o restante para validação
        validation_items = group.iloc[1:]
        validation_data = pd.DataFrame({
            'ITEM': validation_items['ITEM'].tolist(),
            'CATEGORIA': validation_items['CATEGORIA'].tolist()
        })
        validation_df = pd.concat([validation_df, validation_data], ignore_index=True)

# Aplicar a divisão para cada grupo CATEGORIA+PDM
sample_df.groupby(['CATEGORIA', 'PDM']).apply(split_group)

# Exibir informações sobre os conjuntos
print(f"Conjunto de Treinamento: {training_df.shape[0]} itens")
print(f"Conjunto de Validação: {validation_df.shape[0]} itens")

# Mostrar exemplos de ambos os conjuntos
print("\nExemplos do conjunto de treinamento:")
display(training_df.head())

print("\nExemplos do conjunto de validação:")
display(validation_df.head())

# Verificar número de categorias em cada conjunto
print(f"\nNúmero de categorias em treinamento: {training_df['CATEGORIA'].nunique()}")
print(f"Número de categorias em validação: {validation_df['CATEGORIA'].nunique()}")

Conjunto de Treinamento: 10260 itens
Conjunto de Validação: 25399 itens

Exemplos do conjunto de treinamento:


C:\Users\Haroldo Duraes\AppData\Local\Temp\ipykernel_33060\2920078618.py:45: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_df.groupby(['CATEGORIA', 'PDM']).apply(split_group)


,ITEM,CATEGORIA
0,"227505 - ANEL FERROLHO, MATERIAL:AÇO, FUNÇÃO:V...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
1,"227507 - EXTRATOR CÁPSULA, MATERIAL:AÇO CARBON...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
2,"227487 - PINO SEGURANÇA MIRA, MATERIAL:AÇO CAR...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
3,"230202 - MOLA RECUPERADORA, MATERIAL MOLA ESPI...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
4,"359171 - BANDOLEIRA, MATERIAL:CADARÇO DE NYLON...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...



Exemplos do conjunto de validação:


,ITEM,CATEGORIA
0,"227505 - ANEL FERROLHO, MATERIAL:AÇO, FUNÇÃO:V...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
1,"227507 - EXTRATOR CÁPSULA, MATERIAL:AÇO CARBON...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
2,"227487 - PINO SEGURANÇA MIRA, MATERIAL:AÇO CAR...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
3,"230202 - MOLA RECUPERADORA, MATERIAL MOLA ESPI...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
4,"376320 - BANDOLEIRA, MATERIAL:NÁILON, COR:VERD...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...



Número de categorias em treinamento: 460
Número de categorias em validação: 460


In [12]:
# Verificar o formato atual antes da modificação
print("Formato original:")
display(training_df['ITEM'].head())

# Função para extrair apenas a descrição do item (parte após o " - ")
def extrair_descricao(item_string):
    if " - " in item_string:
        return item_string.split(" - ", 1)[1]  # Pega a parte após o primeiro " - "
    return item_string  # Caso não tenha o separador, retorna o texto original

# Aplicar a função aos DataFrames de treinamento e validação
training_df['ITEM'] = training_df['ITEM'].apply(extrair_descricao)
validation_df['ITEM'] = validation_df['ITEM'].apply(extrair_descricao)

# Verificar o resultado após a modificação
print("\nFormato modificado (sem códigos):")
display(training_df['ITEM'].head())

# Mostrar exemplos de ambos os conjuntos após a modificação
print("\nExemplos do conjunto de treinamento após modificação:")
display(training_df.head())

print("\nExemplos do conjunto de validação após modificação:")
display(validation_df.head())

# Verificar as dimensões finais
print(f"\nConjunto de Treinamento: {training_df.shape[0]} itens")
print(f"Conjunto de Validação: {validation_df.shape[0]} itens")

Formato original:


0    227505 - ANEL FERROLHO, MATERIAL:AÇO, FUNÇÃO:V...
1    227507 - EXTRATOR CÁPSULA, MATERIAL:AÇO CARBON...
2    227487 - PINO SEGURANÇA MIRA, MATERIAL:AÇO CAR...
3    230202 - MOLA RECUPERADORA, MATERIAL MOLA ESPI...
4    359171 - BANDOLEIRA, MATERIAL:CADARÇO DE NYLON...
Name: ITEM, dtype: object


Formato modificado (sem códigos):


0    ANEL FERROLHO, MATERIAL:AÇO, FUNÇÃO:VEDAR GASE...
1    EXTRATOR CÁPSULA, MATERIAL:AÇO CARBONO, TIPO:G...
2    PINO SEGURANÇA MIRA, MATERIAL:AÇO CARBONO, FUN...
3    MOLA RECUPERADORA, MATERIAL MOLA ESPIRAL:AÇO F...
4    BANDOLEIRA, MATERIAL:CADARÇO DE NYLON, APLICAÇ...
Name: ITEM, dtype: object


Exemplos do conjunto de treinamento após modificação:


,ITEM,CATEGORIA
0,"ANEL FERROLHO, MATERIAL:AÇO, FUNÇÃO:VEDAR GASE...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
1,"EXTRATOR CÁPSULA, MATERIAL:AÇO CARBONO, TIPO:G...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
2,"PINO SEGURANÇA MIRA, MATERIAL:AÇO CARBONO, FUN...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
3,"MOLA RECUPERADORA, MATERIAL MOLA ESPIRAL:AÇO F...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
4,"BANDOLEIRA, MATERIAL:CADARÇO DE NYLON, APLICAÇ...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...



Exemplos do conjunto de validação após modificação:


,ITEM,CATEGORIA
0,"ANEL FERROLHO, MATERIAL:AÇO, FUNÇÃO:VEDAR GASE...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
1,"EXTRATOR CÁPSULA, MATERIAL:AÇO CARBONO, TIPO:G...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
2,"PINO SEGURANÇA MIRA, MATERIAL:AÇO CARBONO, FUN...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
3,"MOLA RECUPERADORA, MATERIAL MOLA ESPIRAL:AÇO F...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
4,"BANDOLEIRA, MATERIAL:NÁILON, COR:VERDE-OLIVA, ...",10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...



Conjunto de Treinamento: 10260 itens
Conjunto de Validação: 25399 itens


In [16]:
# Exibir alguns exemplos antes da modificação
print("Formato original das categorias:")
display(training_df['CATEGORIA'].head())

# Adicionar a palavra "MATERIAL" no início de cada categoria
training_df['CATEGORIA'] = "MATERIAL; " + training_df['CATEGORIA']
validation_df['CATEGORIA'] = "MATERIAL; " + validation_df['CATEGORIA']

# Verificar o resultado após a modificação
print("\nFormato modificado (com prefixo MATERIAL):")
display(training_df['CATEGORIA'].head())

# Mostrar exemplos de ambos os conjuntos após a modificação
print("\nExemplos do conjunto de treinamento após modificação:")
display(training_df.head())

print("\nExemplos do conjunto de validação após modificação:")
display(validation_df.head())

# Verificar as dimensões finais (não devem mudar)
print(f"\nConjunto de Treinamento: {training_df.shape[0]} itens")
print(f"Conjunto de Validação: {validation_df.shape[0]} itens")

Formato original das categorias:


0    10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
1    10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
2    10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
3    10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
4    10 - ARMAMENTO; 1005 - ARMAS DE FOGO DE CALIBR...
Name: CATEGORIA, dtype: object


Formato modificado (com prefixo MATERIAL):


0    MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
1    MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
2    MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
3    MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
4    MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
Name: CATEGORIA, dtype: object


Exemplos do conjunto de treinamento após modificação:


,ITEM,CATEGORIA
0,"ANEL FERROLHO, MATERIAL:AÇO, FUNÇÃO:VEDAR GASE...",MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
1,"EXTRATOR CÁPSULA, MATERIAL:AÇO CARBONO, TIPO:G...",MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
2,"PINO SEGURANÇA MIRA, MATERIAL:AÇO CARBONO, FUN...",MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
3,"MOLA RECUPERADORA, MATERIAL MOLA ESPIRAL:AÇO F...",MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
4,"BANDOLEIRA, MATERIAL:CADARÇO DE NYLON, APLICAÇ...",MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...



Exemplos do conjunto de validação após modificação:


,ITEM,CATEGORIA
0,"ANEL FERROLHO, MATERIAL:AÇO, FUNÇÃO:VEDAR GASE...",MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
1,"EXTRATOR CÁPSULA, MATERIAL:AÇO CARBONO, TIPO:G...",MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
2,"PINO SEGURANÇA MIRA, MATERIAL:AÇO CARBONO, FUN...",MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
3,"MOLA RECUPERADORA, MATERIAL MOLA ESPIRAL:AÇO F...",MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...
4,"BANDOLEIRA, MATERIAL:NÁILON, COR:VERDE-OLIVA, ...",MATERIAL; 10 - ARMAMENTO; 1005 - ARMAS DE FOGO...



Conjunto de Treinamento: 10260 itens
Conjunto de Validação: 25399 itens


In [17]:
# Definir o caminho para salvar o arquivo Excel
OUT_FILE = CAT_PATH + "catmat_datasets.xlsx"

# Criar um ExcelWriter para salvar múltiplas abas em um único arquivo Excel
with pd.ExcelWriter(OUT_FILE) as writer:
    # Salvar cada DataFrame em uma aba separada
    training_df.to_excel(writer, sheet_name="training", index=False)
    validation_df.to_excel(writer, sheet_name="validation", index=False)
    sample_df.to_excel(writer, sheet_name="sample", index=False)
    


print(f"Datasets salvos em: {OUT_FILE}")
print("Abas salvas:")
print("- training: conjunto de treinamento com prefixo MATERIAL")
print("- validation: conjunto de validação com prefixo MATERIAL")
print("- sample: dataset reduzido por amostragem")

Datasets salvos em: C:\Users\Haroldo Duraes\Desktop\GOvGO\v0\#DATA\PNCP\CAT\catmat_datasets.xlsx
Abas salvas:
- training: conjunto de treinamento com prefixo MATERIAL
- validation: conjunto de validação com prefixo MATERIAL
- sample: dataset reduzido por amostragem
